In [1]:
from FxDerivative import Vanilla
import datetime as dt

In [2]:
option = Vanilla()

c:\Users\S542282\Desktop\fede_env\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
c:\Users\S542282\Desktop\fede_env\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
c:\Users\S542282\Desktop\fede_env\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
c:\Users\S542282\Desktop\fede_env\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
c:\Users\S542282\Desktop\fede_env\Lib\site-packages\scipy\optimize\_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)
c:\Users\S

In [8]:
strike = 1.202
date = dt.date(2026, 7, 3)
option_type = 'call'
N = 100000

In [9]:
option.mkt.spot

1.17685

In [10]:
option.contract(option_type, strike, date, N)
print('Ref:', option.ref, 'option price:', round(option.Premium_EUR(),2), 'EUR')

Ref: 1.17685 option price: 2594.34 EUR


In [11]:
option.delta_EUR()

42060.72827047831